In [5]:
import numpy as np
import os
import uuid

import joblib # for persisting models

# models
from scipy.optimize import curve_fit # multip. linear regression
from sklearn.svm import SVR # support vector forrest
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import pandas as pd
import time
from datetime import datetime

In [6]:
df = pd.read_csv('../aggregated_measurements_data.csv', index_col=0)

df_cleaned = df.dropna()
print("Dropped ", len(df) - len(df_cleaned), " rows with NaN values")

Dropped  780  rows with NaN values


In [7]:
import importlib
import svr_test_bed

importlib.reload(svr_test_bed) # reload exported functions

test_sizes = []
sample_sizes = []
features = []
model_arguments = []

test_sizes.append(0.45)
sample_sizes.append(20000)
features.append(['distance', 'n_power','c_walls', 'w_walls'])
model_arguments.append({'kernel': 'rbf'})

test_sizes.append(0.45)
sample_sizes.append(50000)
features.append(['distance', 'n_power','c_walls', 'w_walls'])
model_arguments.append({'kernel': 'rbf'})

test_specification = pd.DataFrame({
    "test_size": test_sizes,
    "sample_size": sample_sizes,
    "features": features,
    "model_arguments": model_arguments,
})

# assign ids to tests
test_specification['id'] = [str(uuid.uuid4()).replace('-', '')[:8] for _ in range(len(test_specification))]

test_specification['output_file'] = 'results.csv'

(test_results_svr, best_svr_model, best_svr_model_id) = svr_test_bed.svr_test_bed(
    data=df_cleaned,
    test_specification=test_specification,
)

# save best model
joblib.dump(best_svr_model, f"svr_model_{best_svr_model_id}.joblib")

Test 1 of 2 with sample_size 20000, test_size 0.45 and features ['distance', 'n_power', 'c_walls', 'w_walls']
Creating Model
Fitting Model
Predicting
Test 2 of 2 with sample_size 50000, test_size 0.45 and features ['distance', 'n_power', 'c_walls', 'w_walls']
Creating Model
Fitting Model
Predicting
Test took 00:01:58
Best model id:  8a3bd18c  with r2:  0.9940944799115289


['svr_model_8a3bd18c.joblib']

In [8]:
results_df = pd.read_csv('results.csv', index_col=0)
print(results_df)

Empty DataFrame
Columns: []
Index: [0.45, 20000, ['distance', 'n_power', 'c_walls', 'w_walls'], {'kernel': 'rbf'}, 4671d406, results.csv, SVR, 0.45, 50000, ['distance', 'n_power', 'c_walls', 'w_walls'], {'kernel': 'rbf'}, 8a3bd18c, results.csv, SVR]
